In [ ]:
import copy
import random
from torchtrainer.util.train_util import dict_to_argv

# All samples in the VessMAP dataset
names = [
    '5472', '2413', '3406', '7577', '4404', '12005', '10084', '3882', '15577', '15375', '8353', '17035', 
    '13114', '4413', '7783', '11411', '6524', '6581', '13200', '9860', '525', '2643', '8990', '9284', 
    '2050', '2071', '13128', '7865', '14440', '8196', '17880', '1643', '11558', '12943', '2546', '9452', 
    '11828', '8493', '14225', '8256', '1816', '14121', '11161', '16707', '356', '12877', '6818', '10571', 
    '6672', '17702', '15821', '8429', '18180', '13528', '16689', '12960', '5359', '6384', '7392', '6887', 
    '8506', '1585', '4938', '458', '5801', '8686', '15160', '7413', '8065', '8284', '9593', '17584', '2849', 
    '9710', '5740', '4739', '2958', '14787', '11098', '17630', '11111', '6656', '17852', '9000', '12455', '9523', 
    '4909', '12618', '14778', '16295', '17425', '14690', '12749', '12335', '7083', '2287', '482', '7344', '18035', 
    '16766'
]


# Base parameters – these are the defaults for all models.
base_params = {
    # Logging parameters:
    "experiments_path": "/home/fonta42/Desktop/masters-degree/experiments/torch-trainer",  
    "run_name": "",  
    "validate_every": 50,
    "val_img_indices": "0 1 2 3",
    "copy_model_every": 0,
    "wandb_project": "uncategorized",

    # Dataset parameters:
    "dataset_path": "/home/fonta42/Desktop/masters-degree/data/torch-trainer/VessMAP",  
    "dataset_class": "vessmap_few",
    "resize_size": "256 256",        
    "loss_function": "bce",          

    # Model parameters:
    "model_class": "",              

    # Training parameters:
    "num_epochs": 1000, 
    "validation_metric": "Dice",
    "lr": 0.001, 
    "lr_decay": 1.0,
    "bs_train": 2, 
    "bs_valid": 2,
    "weight_decay": 0.0, # TODO: aumentar/variar em casos de overfitting
    "optimizer": "adam", 
    "momentum": 0.9,
    "seed": 42,

    # Device and efficiency parameters:
    "device": "cuda:0",
    "num_workers": 5,
}

# Define variations for the parameters we want to test:
split_variations = [20, 90] # TODO:  variar nos extremos, 0.2 e rand 0.9, garantir que splits tenham tamanhos pares pro batch size bater
val_img_indices_variations = ["0 1 2 3"]
loss_function_variations = ["bce", "cross_entropy"]
lr_variations = [0.001, 0.01]
lr_decay_variations = [1.0, 0.9]
weight_decay_variations = [0.0, 1e-4]

In [ ]:
from medsam_train_torchtrainer import MedsamTrainer

# Override the resize_size and model_class for MedSAM
medsam_base = copy.deepcopy(base_params)
medsam_base["resize_size"] = "1024 1024"
medsam_base["model_class"] = "medsam"
medsam_base["experiment_name"] = "medsam_runs"


for split in split_variations:
    for loss_func in loss_function_variations:
        for lr in lr_variations:
            for lr_decay in lr_decay_variations:
                for wd in weight_decay_variations:
                    params = copy.deepcopy(medsam_base)
                    params['split_strategy'] = ','.join(random.sample(names, split))
                    params["loss_function"] = loss_func
                    params["lr"] = lr
                    params["lr_decay"] = lr_decay
                    params["weight_decay"] = wd
                    # Build the run_name from key parameters:
                    params["run_name"] = f"medsam_{split}_1024x1024_{loss_func}_10_{lr}_{params['bs_train']}_{params['bs_valid']}_{wd}"
                    
                    print("Running MedSAM experiment with parameters:")
                    commandline = ' '.join(dict_to_argv(params, ["dataset_path", "dataset_class", "model_class"]))
                    print(f"python medsam_train_torchtrainer.py {commandline}")
                    #!python medsam_train_torchtrainer.py {commandline}
                    break


In [ ]:
from unet_train import UnetVessTrainer

for split in split_variations:
    for val_indices in val_img_indices_variations:
            for loss_func in loss_function_variations:
                for lr in lr_variations:
                    for lr_decay in lr_decay_variations:
                        for wd in weight_decay_variations:
                            params = copy.deepcopy(base_params)
                            params['split_strategy'] = ','.join(random.sample(names, split))
                            params["val_img_indices"] = val_indices
                            params["loss_function"] = loss_func
                            params["lr"] = lr
                            params["lr_decay"] = lr_decay
                            params["weight_decay"] = wd
                            params["resize_size"] = "256 256"
                            params["model_class"] = "unet_smp"
                            params["run_name"] = f"unet_{split}_256x256_{loss_func}_{params['num_epochs']}_{lr}_{params['bs_train']}_{params['bs_valid']}_{wd}"
                            
                            print("Running UNet experiment with parameters:")
                            print(params)
                            commandline = ' '.join(dict_to_argv(params, ["dataset_path", "dataset_class", "model_class"]))

                            !python unet_train_torchtrainer.py {commandline}


In [3]:
import copy
import random
import itertools
import subprocess # Using subprocess is generally more robust than '!' for running scripts
from torchtrainer.util.train_util import dict_to_argv # Assuming this function is available

# --- Dataset Sample Names ---
# All samples in the VessMAP dataset
names = [
    '5472', '2413', '3406', '7577', '4404', '12005', '10084', '3882', '15577', '15375', '8353', '17035',
    '13114', '4413', '7783', '11411', '6524', '6581', '13200', '9860', '525', '2643', '8990', '9284',
    '2050', '2071', '13128', '7865', '14440', '8196', '17880', '1643', '11558', '12943', '2546', '9452',
    '11828', '8493', '14225', '8256', '1816', '14121', '11161', '16707', '356', '12877', '6818', '10571',
    '6672', '17702', '15821', '8429', '18180', '13528', '16689', '12960', '5359', '6384', '7392', '6887',
    '8506', '1585', '4938', '458', '5801', '8686', '15160', '7413', '8065', '8284', '9593', '17584', '2849',
    '9710', '5740', '4739', '2958', '14787', '11098', '17630', '11111', '6656', '17852', '9000', '12455', '9523',
    '4909', '12618', '14778', '16295', '17425', '14690', '12749', '12335', '7083', '2287', '482', '7344', '18035',
    '16766'
]

# --- Base Parameters ---
# These parameters are common defaults for all experiments.
base_params = {
    # Logging parameters:
    "experiments_path": "/home/fonta42/Desktop/masters-degree/experiments/torch-trainer",
    "run_name": "", # Will be dynamically generated per run
    "validate_every": 50,
    "copy_model_every": 0,
    "wandb_project": "uncategorized",

    # Dataset parameters:
    "dataset_path": "/home/fonta42/Desktop/masters-degree/data/torch-trainer/VessMAP",
    "dataset_class": "vessmap_few",
    "resize_size": "256 256", # Default, can be overridden by model specifics
    "loss_function": "bce", # Default, can be overridden by variations

    # Model parameters:
    "model_class": "", # To be set specifically for each model type

    # Training parameters:
    "num_epochs": 1000,
    "validation_metric": "Dice",
    "lr": 0.001, # Default, can be overridden by variations
    "lr_decay": 1.0, # Default, can be overridden by variations
    "bs_train": 2,
    "bs_valid": 2,
    "weight_decay": 0.0, # Default, can be overridden by variations
    "optimizer": "adam",
    "momentum": 0.9,
    "seed": 42,

    # Device and efficiency parameters:
    "device": "cuda:0",
    "num_workers": 5,
    "benchmark": "", # Empty string likely means 'use benchmark if available' or default behavior
}


# --- Parameter Variations ---
# Define the parameters to vary and their possible values.
# A temporary key 'split_size' is used to represent the number of samples for the split.
# The actual 'split_strategy' string will be generated dynamically for each run.
parameter_variations = {
    "split_strategy": [20, 90],  # Number of samples to select for the training split strategy
    "val_img_indices": ["0 1 2 3", "0 2 4"], # Note: MedSAM loop might ignore this
    "loss_function": ["bce", "cross_entropy"],
    "lr": [0.001, 0.01],
    "lr_decay": [1.0, 0.9],
    "weight_decay": [0.0, 1e-4]
}

In [4]:
# --- Function to Generate Parameter Combinations ---
def generate_parameter_combinations(base_params, variations_dict):
    """
    Generates a list of parameter dictionaries, representing all combinations
    of the variations provided.

    Args:
        base_params (dict): Dictionary of default parameters.
        variations_dict (dict): Dictionary where keys are parameter names and
                                values are lists of possible settings.

    Returns:
        list: A list of dictionaries, each representing a unique experiment configuration.
    """
    keys = list(variations_dict.keys())
    value_lists = [variations_dict[key] for key in keys]

    all_combinations = []
    # Use itertools.product to efficiently get the Cartesian product of all value lists
    for value_combination in itertools.product(*value_lists):
        # Start with a fresh copy of the base parameters for each combination
        params = copy.deepcopy(base_params)
        # Create a dictionary for the current combination of varying parameters
        variation_params = dict(zip(keys, value_combination))
        # Update the base parameters with the current variations
        params.update(variation_params)
        all_combinations.append(params)

    print(f"Generated {len(all_combinations)} parameter combinations.")
    return all_combinations

# --- Generate all parameter sets ---
all_params_list = generate_parameter_combinations(base_params, parameter_variations)
total_experiments = len(all_params_list)

# --- Define Keys to Exclude from Command Line Arguments ---
# These keys are handled internally or set via model overrides, not passed directly via command line flags
# derived from the base combination dictionary in the same way for all models.
exclude_argv_common = ["dataset_path", "dataset_class", "model_class", "split_size"]

Generated 64 parameter combinations.


In [ ]:
# --- Run MedSAM Experiments ---
print(f"\n--- Starting MedSAM Experiments ({total_experiments} runs) ---")
medsam_script = "./medsam_train_torchtrainer.py"
medsam_overrides = {
    "resize_size": "1024 1024",
    "model_class": "medsam",
    "experiment_name": "medsam_runs"
}

for i, base_combo_params in enumerate(all_params_list):
    params = copy.deepcopy(base_combo_params)

    # Apply MedSAM specific overrides
    params.update(medsam_overrides)

    # Generate the split_strategy string dynamically using the 'split_size'
    split_size = params['split_strategy']
    params['split_strategy'] = ','.join(random.sample(names, split_size))

    # Construct a unique run_name reflecting the parameters for this MedSAM run
    # Note: Original MedSAM run_name didn't include split strategy details like count, using split_size instead.
    params["run_name"] = f"medsam_{split_size}_{params['resize_size'].replace(' ','x')}_{params['loss_function']}_{params['num_epochs']}_{params['lr']}_{params['bs_train']}_{params['bs_valid']}_{params['weight_decay']}"

    print(f"\nRunning MedSAM experiment {i+1}/{total_experiments}: {params['run_name']}")
    # print("Parameters:", params) # Uncomment for detailed parameter debugging

    print("Running MedSAM experiment with:")
    commandline = ' '.join(dict_to_argv(params, ["dataset_path", "dataset_class", "model_class"]))
    print(f"python {medsam_script} {commandline}")
    !python {medsam_script} {commandline}

In [ ]:
# --- Run U-Mamba Experiments ---
print(f"\n--- Starting U-Mamba Experiments ({total_experiments} runs) ---")
umamba_script = "./umamba_train_torchtrainer.py"
umamba_overrides = {
    "resize_size": "256 256",
    "model_class": "umamba",
    "experiment_name": "umamba_runs"
}
umamba_exclude_argv = exclude_argv_common # U-Mamba uses val_img_indices

for i, base_combo_params in enumerate(all_params_list):
    params = copy.deepcopy(base_combo_params)
    params.update(umamba_overrides)

    split_size = params['split_strategy']
    params['split_strategy'] = ','.join(random.sample(names, split_size))

    # Construct U-Mamba run_name, potentially including val_img_indices
    val_indices_str = params['val_img_indices'].replace(' ','') # Format val indices for filename
    params["run_name"] = f"umamba_{split_size}_{params['resize_size'].replace(' ','x')}_{params['loss_function']}_{params['num_epochs']}_{params['lr']}_{params['bs_train']}_{params['bs_valid']}_{params['weight_decay']}_val{val_indices_str}"

    print(f"\nRunning U-Mamba experiment {i+1}/{total_experiments}: {params['run_name']}")
    # print("Parameters:", params) # Uncomment for detailed parameter debugging

    print("Running U-Mamba experiment with:")
    commandline = ' '.join(dict_to_argv(params, ["dataset_path", "dataset_class", "model_class"]))
    print(f"python {umamba_script} {commandline}")
    !python {umamba_script} {commandline}


In [ ]:
# --- Run U-Net Experiments ---
print(f"\n--- Starting U-Net Experiments ({total_experiments} runs) ---")
unet_script = "unet_train_torchtrainer.py" # Assuming this is the correct script name
unet_overrides = {
    "resize_size": "256 256",
    "model_class": "unet_smp", # As used in the original code
    "experiment_name": "unet_runs"
}
unet_exclude_argv = exclude_argv_common # U-Net uses val_img_indices

for i, base_combo_params in enumerate(all_params_list):
    params = copy.deepcopy(base_combo_params)
    params.update(unet_overrides)

    split_size = params['split_strategy']
    params['split_strategy'] = ','.join(random.sample(names, split_size))

    # Construct U-Net run_name
    val_indices_str = params['val_img_indices'].replace(' ','')
    params["run_name"] = f"unet_{split_size}_{params['resize_size'].replace(' ','x')}_{params['loss_function']}_{params['num_epochs']}_{params['lr']}_{params['bs_train']}_{params['bs_valid']}_{params['weight_decay']}_val{val_indices_str}"

    print(f"\nRunning U-Net experiment {i+1}/{total_experiments}: {params['run_name']}")
    # print("Parameters:", params) # Uncomment for detailed parameter debugging

    print("Running U-Net experiment with:")
    commandline = ' '.join(dict_to_argv(params, ["dataset_path", "dataset_class", "model_class"]))
    print(f"python unet_train_torchtrainer.py {commandline}")
    !python {unet_script} {commandline}


print("\n--- All experiments finished ---")


--- Starting U-Net Experiments (64 runs) ---

Running U-Net experiment 1/64: unet_20_256x256_bce_1000_0.001_2_2_0.0_val0123
Running U-Net experiment with:
python unet_train_torchtrainer.py /home/fonta42/Desktop/masters-degree/data/torch-trainer/VessMAP vessmap_few unet_smp --experiments_path /home/fonta42/Desktop/masters-degree/experiments/torch-trainer --run_name unet_20_256x256_bce_1000_0.001_2_2_0.0_val0123 --validate_every 50 --copy_model_every 0 --wandb_project uncategorized --resize_size 256 256 --loss_function bce --num_epochs 1000 --validation_metric Dice --lr 0.001 --lr_decay 1.0 --bs_train 2 --bs_valid 2 --weight_decay 0.0 --optimizer adam --momentum 0.9 --seed 42 --device cuda:0 --num_workers 5 --benchmark --split_strategy 4404,9284,4938,12455,5740,5801,16707,11411,8353,9860,17035,10571,13528,2849,8429,1816,11161,2546,7413,13114 --val_img_indices 0 1 2 3 --experiment_name unet_runs
Setting up the experiment...
Done setting up.
Training has started
Training:  80%|███████████